# RAG PDF Chatbot

A multi-turn conversational AI backend powered by Retrieval-Augmented Generation (RAG). Users can upload PDF documents and interact with them through chat.


## Setup 

###  LangSmith Tracing
After signing up at LangSmith, make sure to set your environment variable to start logging traces:
```
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```
or, you can set them manually in a notebook


In [1]:
import getpass 
import os

os.environ["LANGSMITH_TRACING"] = "true" 
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


### Get `.env` variable for LLM API Key


In [2]:
from dotenv import load_dotenv

load_dotenv() 

gemini_api_key = os.getenv("GEMINI_API_KEY")

if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY not found. Please set it up in the .env file")

### Initialize the model

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI 

model = ChatGoogleGenerativeAI(
    model = "gemin-2.5-flash", 
    temperature = 0, 
    api_key = gemini_api_key
)

### Select embeddings model 

I've chosen Gemini embedding, you can choose others like OpenAI, MistralAI or Hugging Face sentnce-transformer models.

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service_account.json"

In [6]:
import json
import os

# Load the project ID dynamically from the credentials file
with open(os.environ["GOOGLE_APPLICATION_CREDENTIALS"], "r") as f:
    credentials = json.load(f)
    project_id = credentials["project_id"]

# Now use it when initializing the embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    project=project_id
)

### Select vector store

I am using FAISS but you can choose others such as Pinecone, Qdrant

In [13]:
import faiss 
from langchain_community.docstore.in_memory import InMemoryDocstore 
from langchain_community.vectorstores import FAISS 

embedding_dim = len(embeddings.embed_query("hello world")) 
index = faiss.IndexFlatL2(embedding_dim) 

vector_store = FAISS(embedding_function=embeddings, 
                    index=index, 
                    docstore=InMemoryDocstore(), 
                    index_to_docstore_id={}, 
                    )